In [1]:
import qonnx
import onnx
from qonnx.core.modelwrapper import ModelWrapper
import hls4ml
from qonnx.transformation.channels_last import ConvertToChannelsLastAndClean
from qonnx.transformation.gemm_to_matmul import GemmToMatMul
from qonnx.util.cleanup import cleanup_model
import plotting

In [2]:
ONNX_PATH = r"onnx\tooth.onnx"
ONNX_PATH_NEW = r"onnx\prun0.75_static_quantized_model_new.onnx"
ONNX_PATH_CLEAN = r"onnx\prun0.75_static_quantized_model_clean_v2.onnx"
TARGET_FPGA_DEVICE = "XCZU7EV-2FFVC1156"

### 找出相同名稱的節點

In [ ]:
import onnx

def inspect_value_info(onnx_model_path, tensor_name):
    model = onnx.load(onnx_model_path)
    graph = model.graph

    print(f"Inspecting ValueInfo for tensor: {tensor_name}")

    # 檢查輸入、輸出和中間值
    for value_info in graph.input:
        if value_info.name == tensor_name:
            print("Found in inputs:", value_info)

    for value_info in graph.output:
        if value_info.name == tensor_name:
            print("Found in outputs:", value_info)

    for value_info in graph.value_info:
        if value_info.name == tensor_name:
            print("Found in value_info:", value_info)

inspect_value_info(ONNX_PATH, "output0")


In [8]:
import onnx

def clean_value_info(onnx_model_path, output_path, tensor_name):
    model = onnx.load(onnx_model_path)
    graph = model.graph

    # 過濾多餘的 value_info，所以不會刪除到 graph.output 的 output0
    new_value_info = [
        value_info for value_info in graph.value_info if value_info.name != tensor_name
    ]
    graph.ClearField("value_info")  # 清空舊的 value_info
    graph.value_info.extend(new_value_info)  # 添加過濾後的條目

    # 保存修復後的模型
    onnx.save(model, output_path)
    print(f"Cleaned model saved to {output_path}")

clean_value_info(ONNX_PATH, ONNX_PATH_NEW, "output0")


Cleaned model saved to onnx\prun0.75_static_quantized_model_new.onnx


### Clean Model
https://fastmachinelearning.org/hls4ml/frontend/qonnx.html  
https://qonnx.readthedocs.io/en/latest/overview.html#from-onnx-to-qonnx

In [19]:
model = ModelWrapper(ONNX_PATH_NEW)
model = cleanup_model(model)
model = model.transform(ConvertToChannelsLastAndClean())
model = model.transform(GemmToMatMul())
model = cleanup_model(model)
model.save(ONNX_PATH_CLEAN)

### Convert ONNX to hls4ml
1. ERROR: Unsupported operation type: QuantizeLinear
   C:\Users\user\anaconda3\envs\YOLOv8_env\Lib\site-packages\hls4ml\converters\onnx\reshape.py
2. Exception: ERROR: Merging more than two tensors is not yet supported.
   C:\Users\user\anaconda3\envs\YOLOv8_env\Lib\site-packages\hls4ml\converters\onnx\merge.py

In [3]:
model = onnx.load(ONNX_PATH)
opset_import = onnx.helper.make_operatorsetid("qonnx.custom_op.channels_last", 1) # 添加 opset_import，確保 QONNX 自定義 OP 可用
model.opset_import.append(opset_import)
model = onnx.shape_inference.infer_shapes(model)

config = hls4ml.utils.config.config_from_onnx_model(
    model, granularity='name', backend='Vitis', default_precision='fixed<16,6>'
)
config['Model']['Precision'] = 'ap_fixed<16,6>'
config['Model']['ReuseFactor'] = 1
print("-----------------------------------")
print("Configuration")
plotting.print_dict(config)
print("-----------------------------------")

hls_model = hls4ml.converters.convert_from_onnx_model(
    model=model,
    output_dir='OutputDir/',
    io_type='io_stream',
    backend='Vitis',
    hls_config=config,
)

-----------------------------------
Configuration
Model
  Precision:         ap_fixed<16,6>
  ReuseFactor:       1
  Strategy:          Latency
-----------------------------------
Interpreting Model ...
Output layers:  ['/model.22/Concat_5']
Input shape: [None, 3, 640, 640]
Topology:
Layer name: /model.0/conv/Conv, layer type: Conv2D, current shape: [[None, 3, 640, 640]]
Layer name: /model.0/act/Sigmoid, layer type: Activation, current shape: [[None, 16, 320, 320]]
Layer name: /model.0/act/Mul, layer type: Merge, current shape: [[None, 16, 320, 320]]
Layer name: /model.1/conv/Conv, layer type: Conv2D, current shape: [[None, 16, 320, 320]]
Layer name: /model.1/act/Sigmoid, layer type: Activation, current shape: [[None, 32, 160, 160]]
Layer name: /model.1/act/Mul, layer type: Merge, current shape: [[None, 32, 160, 160]]
Layer name: /model.2/cv1/conv/Conv, layer type: Conv2D, current shape: [[None, 32, 160, 160]]
Layer name: /model.2/cv1/act/Sigmoid, layer type: Activation, current shape:

IndexError: list index (0) out of range

In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

In [ ]:
hls_model.compile()

In [ ]:

hls_model.build(csim=False)

In [ ]:
hls4ml.report.read_vivado_report('model_1/hls4ml_prj/')

### ChipSkywalker

In [11]:
# Load
onnx_model = onnx.load(ONNX_PATH)
# onnx.checker.check_model(onnx_model)

In [12]:
hls_config = hls4ml.utils.config.config_from_onnx_model(
    onnx_model, granularity='name', backend='Vitis', default_precision='fixed<16,6>'
)
hls_config['Model']['Precision'] = 'ap_fixed<16,6>'
hls_config['Model']['ReuseFactor'] = 1
for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Strategy'] = 'Latency'
    hls_config['LayerName'][Layer]['ReuseFactor'] = 1
hls_config['LayerName']['output_softmax']['Strategy'] = 'Stable'

cfg = hls4ml.converters.create_config(backend='Vitis')  # chipSkywalker modified to use Vitis HLS
cfg['IOType'] = 'io_stream'  # Must set this if using CNNs!
cfg['HLSConfig'] = hls_config
cfg['OnnxModel'] = onnx_model
cfg['OutputDir'] = 'OutputDir/'
cfg['Part'] = TARGET_FPGA_DEVICE   # chipSkywalker use Target Device set at beginning


hls_model = hls4ml.converters.onnx_to_hls(cfg)
hls_model.compile()

KeyError: 'LayerName'

In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

In [ ]:
hls4ml.model.profiling.numerical(model=model, hls_model=hls_model)